In [ ]:
from huggingface_hub import notebook_login
!pip install datasets --quiet
from datasets import load_dataset, load_from_disk, Dataset, DatasetDict
import numpy as np
import pandas as pd
import torch
import unicodedata
#from torch.utils.data import Dataset
import re
import warnings
warnings.filterwarnings("ignore")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
notebook_login()

In [ ]:
df = load_dataset("thu-coai/esconv")

README.md:   0%|          | 0.00/510 [00:00<?, ?B/s]

train.txt:   0%|          | 0.00/4.04M [00:00<?, ?B/s]

valid.txt:   0%|          | 0.00/865k [00:00<?, ?B/s]

test.txt:   0%|          | 0.00/881k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/910 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/195 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/195 [00:00<?, ? examples/s]

In [ ]:
train = [eval(i) for i in df["train"].to_dict()["text"]]
val = [eval(i) for i in df["validation"].to_dict()["text"]]
train = train + val
test = [eval(i) for i in df["test"].to_dict()["text"]]

In [ ]:
def explode_conversation(data):
    '''
    Loop through data, explode dialog to create 1 row per conversation turn between user and counselor.
    Include meta data for each row as well as conversation history

    Reference: https://github.com/thu-coai/Emotional-Support-Conversation

    Input:
        data (dict): json of ESConv. Has following fields:
            - experience_type - user indicates if currently experiencing issue being brought up or issue was in the past.
            - emotion_type - user indicates main emotion they feel going into session.
            - problem_type - user indicates topic of conversation.
            - situation - brief overview of current situation being brought up in session.
            - survey_score - optional survey user fills out after session (scores are between 1-5)
                - initial_emotion_intensity - initial intensity of emotion before session
                - final_emotion_intensity - final intensity of emotion after session
                - empathy - how empathetic and understanding counselor was during session
                - relevance - how relevant the counselor's responses were during session
            - dialog - dictionary of the back and forth conversation between the user and counselor.
                - speaker - "usr" for user and "sys" for counselor
                - text - message from the speaker
                - strategy - only if it's the counselor, the strategy used to respond to user
            - seeker_question1 - feedback from user
            - seeker_question2 - second feedback from user
            - supporter_question1 - feedback from counselor
            - supporter_question2' - second feedback from counselor
    Output:
        convo_turn (dataframe): Exploded dataframe with each row as a turn from the user and counselor. Has following fields:
            - convo_id - conversation ID number, each turn belonging to the same conversation will have the same ID
            - num_turns - number of conversation turns at the current dialog
            - emotion_type - same as input
            - problem_type - same as input
            - situation - same as input
            - emotion_intensity_initial - same as input
            - emotion_intensity_final - same as input
            - empathy - same as input
            - relevance - same as input
            - user_history - Combined text of all the user's messages from the start of the conversation including the current turn
            - user_text - the user's message at the current turn
            - counselor_first - Counselor's first response to the user at the turn
            - counselor_full - Counselor's aggregated response to the user at the turn (if counselor sends multiple messages)
            - strategy - Aggregated strategies counselor used at the current turn
    '''
    # Initialize convo id and list to store conversation turns
    i = 1
    convo_turn = []
    for convo in data:

        emotion_type = convo["emotion_type"]
        problem_type = convo["problem_type"]
        situation = convo["situation"]
        # Survey is optional, might be missing some responses so use get to prevent key error
        emotion_intensity_initial = convo["survey_score"]["seeker"].get("initial_emotion_intensity")
        emotion_intensity_final = convo["survey_score"]["seeker"].get("final_emotion_intensity")
        empathy = convo["survey_score"]["seeker"].get("empathy")
        relevance = convo["survey_score"]["seeker"].get("relevance")

        # Initialize user and counselor text responses and counselor strategies
        prev_speaker = "usr"
        user_text = ""
        user_text_all = ""
        counselor_text_first = ""
        counselor_text_full = ""
        prev_strategy = ""
        n_turns = 0
        for turn in convo["dialog"]:
            # Current speaker and text
            turn_text = turn["text"]
            turn_speaker = turn["speaker"]
            assert turn_speaker in ["usr", "sys"], f"speaker label not usr or sys: {turn_speaker}"
            # Only counselor will have strategy
            turn_strategy = turn.get("strategy")

            # If previous speaker is the counselor and current speaker is the user, indicates end of a conversation turn
            # Add conversation turn to convo_turn list and reset
            if (prev_speaker == "sys") and (turn_speaker == "usr"):
                n_turns += 1
                convo_turn.append({"convo_id": str(i),
                                   "num_turns": n_turns,
                                    "emotion_type": emotion_type,
                                    "problem_type": problem_type,
                                    "situation": situation,
                                    "emotion_intensity_initial": emotion_intensity_initial,
                                    "emotion_intensity_final": emotion_intensity_final,
                                    "empathy": empathy,
                                    "relevance": relevance,
                                    "user_history": user_text_all,
                                    "user_text": user_text,
                                    "counselor_first": counselor_text_first,
                                    "counselor_full": counselor_text_full,
                                    "strategy": prev_strategy})
                # Reset to initial state
                prev_speaker = "usr"
                user_text = ""
                counselor_text_first = ""
                counselor_text_full = ""
                prev_strategy = ""
            # If previous and current speaker is the user, user has sent multiple consecutive texts, combine into 1 text
            if (prev_speaker == "usr") and (turn_speaker == "usr"):
                user_text_all = " ".join([user_text_all, turn_text])
                user_text = " ".join([user_text, turn_text])
                continue
            # If previous speaker is user and current speaker is counselor,
            # indicates it's counselor's first response to user after user finished messaging
            if (prev_speaker == "usr") and (turn_speaker == "sys"):
                counselor_text_first = " ".join([counselor_text_first, turn_text])
                counselor_text_full = " ".join([counselor_text_full, turn_text])
                prev_speaker = "sys"
                prev_strategy = turn_strategy
                continue
            # If previous speaker is counselor and current speaker is counselor, counselor sent multiple consecutive texts
            # Keep track of text and strategies separately
            if (prev_speaker == "sys") and (turn_speaker == "sys"):
                counselor_text_full = " ".join([counselor_text_full, turn_text])
                prev_strategy = ", ".join([prev_strategy, turn_strategy])
                continue
        # Get the last conversation turn only if it's complete (both user and counselor speak)
        turn_speaker = "usr"
        if (prev_speaker == "sys") and (turn_speaker == "usr"):
            n_turns += 1
            convo_turn.append({"convo_id": str(i),
                               "num_turns": n_turns,
                                "emotion_type": emotion_type,
                                "problem_type": problem_type,
                                "situation": situation,
                                "emotion_intensity_initial": emotion_intensity_initial,
                                "emotion_intensity_final": emotion_intensity_final,
                                "empathy": empathy,
                                "relevance": relevance,
                                "user_history": user_text_all,
                                "user_text": user_text,
                                "counselor_first": counselor_text_first,
                                "counselor_full": counselor_text_full,
                                "strategy": prev_strategy})

        i += 1

    return pd.DataFrame(convo_turn)

def normalize_text(text):
    return unicodedata.normalize('NFKD', text.replace("ð", "d")) \
                                  .encode('ascii', 'ignore').decode('utf-8')

def filter_reformat(df):
    '''
    Filter to just conversations rated as high empathy (score of 4 or 5).
    Filter to conversations in rated as relevant (score of 4 or 5). (Don't want to risk hallucinations)
    Remove first pass if it was initiated by the counselor and adjust num_turns variable accordingly (Chatbot will be responsive)
    Additional reformatting of types
    Input:
        df (dataframe): output from explode_conversation function
    Output:
        df (dataframe): filtered and reformatted input
    '''
    # Identify conversations in which counselor started conversation
    # reduce number of turns by 1 and remove counselor initiated rows
    counselor_start_convos = df[df["user_text"] == ""]["convo_id"]
    df.loc[df["convo_id"].isin(counselor_start_convos), "num_turns"] = df.loc[df["convo_id"].isin(counselor_start_convos), "num_turns"] - 1
    df = df[#(df["empathy"].isin(["4", "5"])) &
           #(df["relevance"].isin(["4", "5"])) &
           (df["user_text"] != "")]
    # Only remove extra whitespace, LLM should be able to handle misspellings
    df["user_text"] = df["user_text"].str.replace("\s+", " ").str.strip()
    df["user_history"] = df["user_history"].str.replace("\s+", " ").str.strip()
    df["counselor_first"] = df["counselor_first"].str.replace("\s+", " ").str.strip()
    df["counselor_full"] = df["counselor_full"].str.replace("\s+", " ").str.strip()
    # recode utf-8 characters that prevent from pushing to huggingface
    df["situation"] = df["situation"].apply(lambda text: normalize_text(text))
    df["user_history"] = df["user_history"].apply(lambda text: normalize_text(text))
    df["user_text"] = df["user_text"].apply(lambda text: normalize_text(text))
    df["counselor_first"] = df["counselor_first"].apply(lambda text: normalize_text(text))
    df["counselor_full"] = df["counselor_full"].apply(lambda text: normalize_text(text))
    # Split strategies and take the first strategy to pair with counselor's first message in turn
    df["first_strategy"] = df["strategy"].str.split(", ").apply(lambda x: x[0])
    # lower and replace spaces with underscores for problem type reformat survey scores to float
    df["problem_type"] = df["problem_type"].str.lower().str.replace("\s", "_", regex = True)
    df["emotion_intensity_initial"] = df["emotion_intensity_initial"].astype(float)
    df["emotion_intensity_final"] = df["emotion_intensity_final"].astype(float)
    df["empathy"] = df["empathy"].astype(float)
    df["relevance"] = df["relevance"].astype(float)
    return df

In [ ]:
train_turns = explode_conversation(train)
test_turns = explode_conversation(test)

In [ ]:
train_reformatted = filter_reformat(train_turns)
test_reformatted = filter_reformat(test_turns)

In [ ]:
# train_dataset = Dataset.from_pandas(train_reformatted)
# test_dataset = Dataset.from_pandas(test_reformatted)

# dataset_dict = DatasetDict({
#     'train': train_dataset,
#     'test': test_dataset
# })

# dataset_dict.push_to_hub("jordanfan/esconv_processed")

In [ ]:
# Remove rows in which there is no empathy or relevance score (there are no cases in which there is a value for empathy and no value for relevance or vice versa)
train_reformatted = train_reformatted.dropna(subset = 'empathy')
train_reformatted['empathy_relevance'] = train_reformatted.apply(lambda x: 'high empathy' if x['empathy'] >= 4 else 'low empathy', axis = 1)
train_reformatted['empathy_relevance'] = train_reformatted.apply(lambda x: ', '.join([x['empathy_relevance'], 'high relevance']) if x['relevance'] >= 4
                                                                else ', '.join([x['empathy_relevance'], 'low relevance']), axis = 1)
test_reforamtted = test_reformatted.dropna(subset = 'empathy')
test_reformatted['empathy_relevance'] = test_reformatted.apply(lambda x: 'high empathy' if x['empathy'] >= 4 else 'low empathy', axis = 1)
test_reformatted['empathy_relevance'] = test_reformatted.apply(lambda x: ', '.join([x['empathy_relevance'], 'high relevance']) if x['relevance'] >= 4
                                                                else ', '.join([x['empathy_relevance'], 'low relevance']), axis = 1)
train_dataset = Dataset.from_pandas(train_reformatted)
test_dataset = Dataset.from_pandas(test_reformatted)

dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

dataset_dict.push_to_hub("jordanfan/esconv_processed_all")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jordanfan/esconv_processed_all/commit/dded282a00c6672b452791a8949eb89951e5a688', commit_message='Upload dataset', commit_description='', oid='dded282a00c6672b452791a8949eb89951e5a688', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jordanfan/esconv_processed_all', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jordanfan/esconv_processed_all'), pr_revision=None, pr_num=None)

## Create user emotion column for each pass

In [ ]:
df = load_dataset("jordanfan/esconv_processed")

README.md:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.27M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/611k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9588 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1865 [00:00<?, ? examples/s]

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

unique_emotions = ['angry', 'disgusted', 'fear', 'happy', 'neutral', 'sad', 'surprised',]

model_name = 'MatthewFrank/roberta-large_pytorch_ALlData_V01'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(unique_emotions))

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
def get_emotion_predictions(df):
  # Tokenize the input statements
  model_inputs = tokenizer(df["user_text"], padding=True, truncation=True, return_tensors="pt")
  # Put the model in evaluation mode
  model.eval()
  # Run inference
  with torch.no_grad():
    outputs = model(**model_inputs)
  # Get the logits (raw predictions before softmax)
  logits = outputs.logits
  # Convert logits to probabilities using softmax
  probabilities = torch.nn.functional.softmax(logits, dim=-1)
  # Map each input to its predicted label
  prediction = torch.argmax(probabilities, dim=1)
  anger_proba = probabilities[0][0].item()
  disgusted_proba = probabilities[0][1].item()
  fear_proba = probabilities[0][2].item()
  happy_proba = probabilities[0][3].item()
  neutral_proba = probabilities[0][4].item()
  sad_proba = probabilities[0][5].item()
  surprised_proba = probabilities[0][6].item()
  return {"user_turn_emotion": unique_emotions[prediction[0]],
          "anger_proba": anger_proba,
          "disgusted_proba": disgusted_proba,
          "fear_proba": fear_proba,
          "happy_proba": happy_proba,
          "neutral_proba": neutral_proba,
          "sad_proba": sad_proba,
          "surprised_proba": surprised_proba}

In [ ]:
df = df.map(get_emotion_predictions)

Map:   0%|          | 0/9588 [00:00<?, ? examples/s]

Map:   0%|          | 0/1865 [00:00<?, ? examples/s]

In [ ]:
df.push_to_hub("jordanfan/esconv_processed")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jordanfan/esconv_processed/commit/feb9b3ff1a1addba075f2fa55b87555b3c8f8236', commit_message='Upload dataset', commit_description='', oid='feb9b3ff1a1addba075f2fa55b87555b3c8f8236', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jordanfan/esconv_processed', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jordanfan/esconv_processed'), pr_revision=None, pr_num=None)

In [ ]:
# Export to csv for EDA
from google.colab import drive
drive.mount('/content/drive/')
df["train"].to_pandas().drop(columns = ["situation", "user_history", "user_text", "counselor_first", "counselor_full", "__index_level_0__"], axis = 1).to_csv("/content/drive/MyDrive/Colab Notebooks/W210/esconv_processed.csv")

Mounted at /content/drive/


## Create user emotion for each pass for low empathy/low relevance examples

In [ ]:
df = load_dataset("jordanfan/esconv_processed")
df_all = load_dataset("jordanfan/esconv_processed_all")

README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.76M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/764k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11107 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2295 [00:00<?, ? examples/s]

In [ ]:
df_low_empathy_relevance = df_all.filter(lambda example: example["empathy_relevance"] in ["low empathy, high relevance", "low empathy, low relevance", "high empathy, low relevance"])

Filter:   0%|          | 0/11107 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2295 [00:00<?, ? examples/s]

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

unique_emotions = ['angry', 'disgusted', 'fear', 'happy', 'neutral', 'sad', 'surprised',]

model_name = 'MatthewFrank/roberta-large_pytorch_ALlData_V01'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(unique_emotions))

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
def get_emotion_predictions(df):
  # Tokenize the input statements
  model_inputs = tokenizer(df["user_text"], padding=True, truncation=True, return_tensors="pt")
  # Put the model in evaluation mode
  model.eval()
  # Run inference
  with torch.no_grad():
    outputs = model(**model_inputs)
  # Get the logits (raw predictions before softmax)
  logits = outputs.logits
  # Convert logits to probabilities using softmax
  probabilities = torch.nn.functional.softmax(logits, dim=-1)
  # Map each input to its predicted label
  prediction = torch.argmax(probabilities, dim=1)
  anger_proba = probabilities[0][0].item()
  disgusted_proba = probabilities[0][1].item()
  fear_proba = probabilities[0][2].item()
  happy_proba = probabilities[0][3].item()
  neutral_proba = probabilities[0][4].item()
  sad_proba = probabilities[0][5].item()
  surprised_proba = probabilities[0][6].item()
  return {"user_turn_emotion": unique_emotions[prediction[0]],
          "anger_proba": anger_proba,
          "disgusted_proba": disgusted_proba,
          "fear_proba": fear_proba,
          "happy_proba": happy_proba,
          "neutral_proba": neutral_proba,
          "sad_proba": sad_proba,
          "surprised_proba": surprised_proba}

In [ ]:
df_low_empathy_relevance = df_low_empathy_relevance.map(get_emotion_predictions)

Map:   0%|          | 0/1519 [00:00<?, ? examples/s]

Map:   0%|          | 0/430 [00:00<?, ? examples/s]

In [ ]:
df = df.map(lambda x: {"empathy_relevance": "high empathy, high relevance"})

Map:   0%|          | 0/9588 [00:00<?, ? examples/s]

Map:   0%|          | 0/1865 [00:00<?, ? examples/s]

In [ ]:
from datasets import concatenate_datasets
train_datasets = concatenate_datasets([df["train"], df_low_empathy_relevance["train"]])
test_datasets = concatenate_datasets([df["test"], df_low_empathy_relevance["test"]])
concatenated_dataset = DatasetDict({
    'train': train_datasets,
    'test': test_datasets
})

concatenated_dataset.push_to_hub("jordanfan/esconv_processed_all")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jordanfan/esconv_processed_all/commit/6121c28232e70ecb347943bf06af17061895debb', commit_message='Upload dataset', commit_description='', oid='6121c28232e70ecb347943bf06af17061895debb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jordanfan/esconv_processed_all', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jordanfan/esconv_processed_all'), pr_revision=None, pr_num=None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
concatenated_dataset["train"].to_pandas().drop(columns = ["situation", "user_history", "user_text", "counselor_first", "counselor_full", "__index_level_0__"], axis = 1).to_csv("/content/drive/MyDrive/Colab Notebooks/W210/esconv_processed_all.csv")

Mounted at /content/drive/
